In [1]:
import os,sys
import pandas as pd
from PIL import Image
from os import listdir
from numpy import zeros, asarray, expand_dims, mean
import numpy as np

ROOT_DIR = os.path.abspath(r"C:\Users\My PC\Desktop\maskrcnn - py3.8 2\Mask-R-CNN-using-Tensorflow2")
sys.path.append(ROOT_DIR) 

from mrcnn.utils import Dataset,extract_bboxes
from mrcnn.visualize import display_instances
from matplotlib import pyplot
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

import warnings
warnings.filterwarnings("ignore")

In [2]:
class PredictionConfig(Config):
    NAME = "ped_cfg"
    NUM_CLASSES = 1 + 3
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    USE_MINI_MASK = False

In [3]:
cfg = PredictionConfig()
model = MaskRCNN(mode='inference', model_dir=r'C:\Users\My PC\Desktop\maskrcnn - py3.8 2\Mask-R-CNN-using-Tensorflow2\mask_rcnn_fishes_0099.h5', config=cfg)

Instructions for updating:
Use fn_output_signature instead


In [4]:
for i in listdir():
    print(i)
    if i[:3]=='ped':
        path=i

.git
cocoapi
convert_coco_via
custom.ipynb
custom.py
eval.ipynb
gt_pred_test.json
mask_rcnn_coco.h5
mask_rcnn_fishes_0099.h5
merged_via_annotations.json
mrcnn
no-augment
README.md
requirements.txt
test_model.ipynb
validation_per_image_ap50.csv
via_train.json
via_valid.json


In [5]:
model.load_weights(r'C:\Users\My PC\Desktop\maskrcnn - py3.8 2\Mask-R-CNN-using-Tensorflow2\mask_rcnn_fishes_0099.h5', by_name=True)

In [ ]:
# def evaluate_model(dataset, model, cfg):
#     APs = list()
#     j=0
#     for image_id in dataset.image_ids:
#         if j>50:
#             continue
#         image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id)
#         scaled_image = mold_image(image, cfg)
#         sample = expand_dims(scaled_image, 0)
#         yhat = model.detect(sample, verbose=0)
#         r = yhat[0]
#         AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
#         APs.append(AP)
#         j+=1
#     mAP = mean(APs)
#     return mAP

In [22]:
pip install pycocotools-windows


Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np
from mrcnn.model import load_image_gt, mold_image
from mrcnn.utils import compute_ap

def evaluate_model(dataset, model, cfg, iou_thresholds=None, max_images=50):
    """
    dataset: Mask R-CNN Dataset object
    model: trained Mask R-CNN model
    cfg: model config
    iou_thresholds: list of IoU thresholds (default = np.arange(0.5, 1.0, 0.05))
    max_images: number of images to evaluate (default = 50)
    """

    if iou_thresholds is None:
        iou_thresholds = np.arange(0.5, 1.0, 0.05)  # COCO standard: 0.5:0.95

    APs = {thr: [] for thr in iou_thresholds}
    precisions, recalls = [], []

    for i, image_id in enumerate(dataset.image_ids):
        if i >= max_images:
            break

        # Load image and ground truth
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id)
        molded_image = mold_image(image, cfg)
        sample = np.expand_dims(molded_image, 0)

        # Run detection
        results = model.detect(sample, verbose=0)
        r = results[0]

        # Compute AP for each IoU threshold
        for thr in iou_thresholds:
            AP, precision, recall, _ = compute_ap(
                gt_bbox, gt_class_id, gt_mask,
                r["rois"], r["class_ids"], r["scores"], r["masks"],
                iou_threshold=thr
            )
            APs[thr].append(AP)
            if thr == 0.5:  # store precision/recall only for IoU=0.5
                precisions.append(np.mean(precision))
                recalls.append(np.mean(recall))

    # Aggregate results
    mean_ap_50 = np.mean(APs[0.5]) if 0.5 in APs else None
    mean_ap_5095 = np.mean([np.mean(APs[thr]) for thr in iou_thresholds])
    mean_precision = np.mean(precisions)
    mean_recall = np.mean(recalls)

    print("Precision (IoU=0.5): %.3f" % mean_precision)
    print("Recall (IoU=0.5): %.3f" % mean_recall)
    print("mAP@0.5: %.3f" % mean_ap_50)
    print("mAP@0.5:0.95: %.3f" % mean_ap_5095)

    return mean_precision, mean_recall, mean_ap_50, mean_ap_5095


In [15]:
import os
import numpy as np
from mrcnn.utils import Dataset
from pycocotools.coco import COCO

class FishCocoDataset(Dataset):
    def load_dataset(self, dataset_dir, subset):
        """
        dataset_dir: root dataset folder
        subset: 'train' or 'valid'
        """

        # Load COCO annotations
        ann_file = os.path.join(dataset_dir, subset, "_annotations.coco.json")
        coco = COCO(ann_file)

        # Add classes (make sure these match your COCO categories)
        self.add_class("dataset", 1, "catfish")
        self.add_class("dataset", 2, "milkfish")
        self.add_class("dataset", 3, "molobicus")

        # Load all images
        image_ids = list(coco.imgs.keys())
        for image_id in image_ids:
            image_info = coco.imgs[image_id]
            path = os.path.join(dataset_dir, subset, image_info["file_name"])

            self.add_image(
                "dataset",
                image_id=image_id,
                path=path,
                annotations=coco.loadAnns(coco.getAnnIds(imgIds=[image_id]))
            )

        self.coco = coco  # store reference for later

    def load_mask(self, image_id):
        """Load instance masks for an image"""
        image_info = self.image_info[image_id]
        annotations = image_info["annotations"]

        masks = []
        class_ids = []
        for ann in annotations:
            class_id = ann["category_id"]
            if class_id:
                # Decode segmentation mask
                mask = self.coco.annToMask(ann)
                masks.append(mask)
                class_ids.append(class_id)

        if masks:
            masks = np.stack(masks, axis=-1)
            class_ids = np.array(class_ids, dtype=np.int32)
            return masks, class_ids
        else:
            # No annotations
            return np.empty([0, 0, 0]), np.array([], dtype=np.int32)

    def image_reference(self, image_id):
        return self.image_info[image_id]["path"]


In [8]:
dataset = r"C:\Users\My PC\Desktop\maskrcnn - py3.8 2\Mask-R-CNN-using-Tensorflow2\no-augment"

In [19]:
# Training dataset
train_set = FishCocoDataset()
train_set.load_dataset(dataset, "train")
train_set.prepare()

# Validation dataset
val_set = FishCocoDataset()
val_set.load_dataset(dataset, "valid")
val_set.prepare()

# Evaluate
# train_mAP = evaluate_model(train_set, model, cfg)
# print("Train mAP: %.3f" % train_mAP)
# val_mAP = evaluate_model(val_set, model, cfg)
# print("Test mAP: %.3f" % val_mAP)

precision, recall, mAP50, mAP5095 = evaluate_model(val_set, model, cfg)


loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Precision (IoU=0.5): 0.562
Recall (IoU=0.5): 0.430
mAP@0.5: 0.489
mAP@0.5:0.95: 0.249


In [14]:
dir "C:\Users\My PC\Desktop\maskrcnn - py3.8 2\Mask-R-CNN-using-Tensorflow2\no-augment\train\images" | findstr test_53


SyntaxError: invalid syntax (3902050483.py, line 1)